# 120+ LOSS WATCH

The 2025 Colorado Rockies are *not* good at baseball. Neither were the 1962 Mets or 2024 White Sox. Each of those two teams lost 120 or 121 games, respectively. The 2025 Rockies are on a path to 120 losses. My goal is to track that throughout the season!

In [1]:
# import the required packages
import pandas as pd
import numpy as np

In [2]:
# read in the schedules for the three teams
rockies = pd.read_csv('../Data/Raw/COL_2025_schedule.csv')
sox = pd.read_csv('../Data/Raw/CHW_2024_schedule.csv', usecols=['Gm#', 'W/L', 'R', 'RA'])
mets = pd.read_csv('../Data/Raw/NYM_1962_schedule.csv', usecols=['Gm#', 'W/L', 'R', 'RA'])

Column names should match.

In [3]:
for team in [sox, mets]:
    team.rename(columns={'Gm#': 'Games Played'}, inplace=True)
    team.rename(columns={'W/L': 'Result'}, inplace=True)

sox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Games Played  162 non-null    int64 
 1   Result        162 non-null    object
 2   R             162 non-null    int64 
 3   RA            162 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 5.2+ KB


In [4]:
mets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Games Played  161 non-null    int64 
 1   Result        161 non-null    object
 2   R             161 non-null    int64 
 3   RA            161 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 5.2+ KB


I will need to combine these dataframes so that I can use them in a dashboard. Each should have a column for the team name.

In [5]:
# add in the team column
rockies['Team'] = '2025 COL'
sox['Team'] = '2024 CHW'
mets['Team'] = '1962 NYM'

rockies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Games Played  31 non-null     int64 
 1   Result        31 non-null     object
 2   R             31 non-null     int64 
 3   RA            31 non-null     int64 
 4   Team          31 non-null     object
dtypes: int64(3), object(2)
memory usage: 1.3+ KB


Each team will need a running total of the number of losses. I would also like to track their expected losses using the Pythagorean Win-Loss record developed by Bill James.

$$ \text{Pythagorean W-L} = \frac{R^2}{R^2 +{RA}^2}$$

In [6]:
teams = [rockies, sox, mets]

for team in teams:
    team['Loss?'] = team['Result'].str.startswith('L').astype('int') # will assign 1 if started with an 'L'
    team['Losses'] = team['Loss?'].cumsum()
    team['Expected Losses'] = round(team['Games Played'] * (1 - team['R'].cumsum() ** 2 / (team['R'].cumsum() ** 2 + team['RA'].cumsum() ** 2))).astype(int)

rockies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Games Played     31 non-null     int64 
 1   Result           31 non-null     object
 2   R                31 non-null     int64 
 3   RA               31 non-null     int64 
 4   Team             31 non-null     object
 5   Loss?            31 non-null     int64 
 6   Losses           31 non-null     int64 
 7   Expected Losses  31 non-null     int64 
dtypes: int64(6), object(2)
memory usage: 2.1+ KB


Looks ready to join together as one!

In [7]:
losers = pd.concat([rockies, sox, mets], ignore_index=True)
losers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Games Played     354 non-null    int64 
 1   Result           354 non-null    object
 2   R                354 non-null    int64 
 3   RA               354 non-null    int64 
 4   Team             354 non-null    object
 5   Loss?            354 non-null    int64 
 6   Losses           354 non-null    int64 
 7   Expected Losses  354 non-null    int64 
dtypes: int64(6), object(2)
memory usage: 22.2+ KB


In [8]:
losers.to_csv('../Data/Processed/losers.csv', index=False)